# wo shi ji ji kun !!!

In [50]:
from lmfit import Parameters
from lmfit import minimize
import numpy as np
import pandas as pd

### Define parameters

In [94]:
# risk aversion
delta = 4  # 3.5 < delta < 4.5

In [95]:
# convariance_quantities
sigma_all = [[.02778, .00387, .00021],
             [.00387, .01112, -.00020],
             [.00021, -.00020, .00115],
            ]

In [135]:
pd.DataFrame(sigma_all)

,0,1,2
0,0.02778,0.00387,0.00021
1,0.00387,0.01112,-0.00020
2,0.00021,-0.00020,0.00115


In [137]:
# expect_return
expect_return = [0.1073, 0.0737, 0.0627]

### Generate intial guess for xi 

In [115]:
# Generate random xi or input yours as list
nbr_assets = len(expect_return)
xi_weights = list(np.random.dirichlet(np.ones(nbr_assets),size=1)[0]) # generate random xi list
if sum(xi_weights)!=1: # test the sum is 1
    raise ValueError("The sum of your input weight '{}' is not '1'.".format(xi_weights))
print(xi_weights)
print("Sum: {}".format(sum(xi_weights)))

[0.88005816182921315, 0.055423928574773233, 0.064517909596013737]
Sum: 1.0


### Form params for minimization

In [140]:
# pack parameters for fitting
params = Parameters()
for n in range(len(xi_weights)):
    params.add('x' + str(n+1),
                value=xi_weights[n],
                vary=True,
                min=0)
print("\n----------Before apply constraint---------- \n")
params.pretty_print()

# form constraint
_param_list = list(params.valuesdict().keys())
_constraint_param = _param_list.pop(-1)
_constraint = '-'.join(_param_list)
_constraint = '1-' + _constraint

# set constraint
params[_constraint_param].set(expr=_constraint)
print("\n----------After apply constraint---------- \n")
params.pretty_print()



----------Before apply constraint---------- 

Name     Value      Min      Max   Stderr     Vary     Expr Brute_Step
x1    0.8801        0      inf     None     True     None     None
x2   0.05542        0      inf     None     True     None     None
x3   0.06452        0      inf     None     True     None     None

----------After apply constraint---------- 

Name     Value      Min      Max   Stderr     Vary     Expr Brute_Step
x1    0.8801        0      inf     None     True     None     None
x2   0.05542        0      inf     None     True     None     None
x3   0.06452        0      inf     None    False  1-x1-x2     None


# Define model

In [147]:
def model(params):
    # unpack params
    parvals = params.valuesdict()
    param_names = parvals.keys()
    xi_weights = []
    for each in param_names:
        xi_weights.append(parvals[each])
    print(xi_weights)
    
    # calculate the cost
    sum_1 = 0
    sum_2 = 0
    for i in range(len(xi_weights)):
        j=0
        _sum_1 = xi_weights[i] * expect_return[i]
        sum_1 = sum_1 + _sum_1
        while j <= len(xi_weights)-1:
            print(i, j)
            _sum_2 = sigma_all[i][j] * xi_weights[i] * xi_weights[j]
            sum_2 = sum_2 + _sum_2
            j=j+1
    cost = sum_1 - (delta/2)*sum_2
    return cost
    

In [146]:
model(params)

[0.88005816182921315, 0.055423928574773233, 0.064517909596013612]
0 0
0 1
0 2
1 0
1 1
1 2
2 0
2 1
2 2


0.058651123722012852

In [152]:
result = minimize(model, params, method='newton',jac=None)

ValueError: Jacobian is required for Newton-CG method